In [2]:
urls = [
    "https://www.has-sante.fr/jcms/r_1502632/fr/diabete-de-type-2",
    "https://www.ameli.fr/assure/sante/themes/diabete/comprendre-diabete",
    "https://www.inserm.fr/dossier/diabete/",
]

from trafilatura import fetch_url

raw_pages = []

for url in urls:
    downloaded = fetch_url(url)
    raw_pages.append({
        "url": url,
        "html": downloaded
    })

raw_pages

[{'url': 'https://www.has-sante.fr/jcms/r_1502632/fr/diabete-de-type-2',
  'html': '\r\n\r\n<!DOCTYPE html>\n<html lang="fr" xml:lang="fr" dir="ltr"   >\n<head>\n  <base href="https://www.has-sante.fr/"  />\n  \n  <meta charset="UTF-8" />\n  <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n  <meta name="Generator" content="Jalios JCMS - Copyright (C) Jalios S.A. 2001 - 2025 - https://www.jalios.com/" />\n  <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n  <meta name="keywords" content="recommandations, évaluation, certification établissements, accréditation médecins, pratiques professionnelles, soins médicaux, ALD, service médical rendu, médicaments, santé, has, haute autorité de santé, certification, assurance maladie, patient, qualité soins, commission transparence" />\n  <meta name="description" content="Autorité publique indépendante à caractère scientifique, la Haute Autorité de santé (HAS) vise à développer la qualité dans le champ sanitaire, soc

In [5]:
from trafilatura import extract, extract_metadata

data = []

for item in raw_pages:
    html = item["html"]
    url = item["url"]

    if html is None:
        continue

    # Texte principal
    texte = extract(html)

    # Métadonnées HTML
    meta = extract_metadata(html)


    # Par défaut
    titre = None
    auteur = None
    date = None
    description = None

    # CAS 1 : meta est un dict
    if isinstance(meta, dict):
        titre = meta.get("title")
        auteur = meta.get("author")
        date = meta.get("date")
        description = meta.get("description")

    # CAS 2 : meta est un Document (objets avec attributs)
    else:
        titre = getattr(meta, "title", None)
        auteur = getattr(meta, "author", None)
        date = getattr(meta, "date", None)
        description = getattr(meta, "description", None)

    data.append({
        "url": url,
        "texte": texte,
        "titre": titre,
        "auteur": auteur,
        "date": date,
        "description": description
    })

data


[{'url': 'https://www.has-sante.fr/jcms/r_1502632/fr/diabete-de-type-2',
  'texte': "Il est primordial, dans l’accompagnement des personnes présentant un trouble du développement intellectuel (TDI), de prendre en compte leurs droits, besoins et préférences. Tel était le message principal porté par la Haute Autorité de santé (HAS) en 2022 dans le premier volet de ses recommandations à destination des professionnels et des proches accompagnant les personnes concernées. La HAS en publie aujourd’hui le second volet, consacré aux différents environnements de vie : la scolarité, le travail et le temps libre. Ces recommandations seront, comme les premières, transcrites en facile à lire et à comprendre (FALC).\nActualités\nApprendre, travailler et profiter de ses loisirs quand on présente un trouble du développement intellectuel\nLa HAS recommande de ne plus prescrire la vitesse de sédimentation\nLa vitesse de sédimentation (VS) est un marqueur de l’inflammation. L’usage de ce test sanguin est

In [6]:
import pandas as pd

df = pd.DataFrame(data)
df

,url,texte,titre,auteur,date,description
0,https://www.has-sante.fr/jcms/r_1502632/fr/dia...,"Il est primordial, dans l’accompagnement des p...",HAS - Professionnels,None,2025-11-01,Autorité publique indépendante à caractère sci...
1,https://www.inserm.fr/dossier/diabete/,AccueilDossiersDiabète de type 1Diabète de typ...,"Diabète de type 1 · Inserm, La science pour la...",Inserm,2017-07-11,Le diabète correspond à une élévation prolongé...


In [7]:
df = df[df["texte"].notnull()] # supprimer les lignes vides
df["texte"] = df["texte"].str.strip() # retirer espace et saut de lignes inutiles
df = df.drop_duplicates(subset="url") # supprimer les doublons par url
df = df.drop_duplicates(subset="texte") # supprimer les doublons ar texts
df["longueur"] = df["texte"].apply(len) # ajouter une longueur de texte


In [8]:
df.to_csv("corpus_diabete_clean.csv", index=False) # sauvegarder le corpus nettoyer